<a href="https://colab.research.google.com/github/GretaDanesi18/Olive_Detection_Tracking/blob/main/YOLOv5_olive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **SETUP**

Per prima cosa cloniamo la repo contente la YOLOv5 e installiamo i requisiti

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
! pip install -r requirements.txt

Importiamo le librerie necessarie

In [2]:
import torch 
import os
import random 
import shutil
from IPython.display import Image
from PIL import Image, ImageDraw
import cv2
import pandas as pd
import subprocess
from numpy import source


Controlliamo l'utilizzo del device GPU oppure CPU

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print('Using PyTorch version:', torch.__version__, ' Device:', device)

## **CREAZIONE DATASET SOURCE DETECTOR**

Cloniamo la repo contente il dataset iniziale, i frame dei video e il file yaml

In [21]:
!git clone https://github.com/GretaDanesi18/Olive_Detection_Tracking.git

Cloning into 'Olive_Detection_Tracking'...
remote: Enumerating objects: 1880, done.
^C


Attraverso la libreria os creiamo le directory in cui inserire le labels e le immagini necessarie a creare il dataset per yolov5
```
/path/to/dataset/...
+--images
|   +---val
         +-----image_1.jpg
         +-----image_2.jpg
         +-----image_n.jpg
|   +---train
        +-----image_1.jpg
        +-----image_2.jpg
        +-----image_n.jpg
+--labels
|   +---val
         +-----label_1.txt
         +-----label_2.txt
         +-----label_n.txt
|   +---train
        +-----label_1.txt
        +-----label_2.txt
        +-----label_n.txt




In [22]:
try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/images/val'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/images/val')

except OSError:
    print('Errore nella creazione della cartella images/val')

try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/images/train'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/images/train')

except OSError:
    print('Errore nella creazione della cartella images/train')

try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/labels/val'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/labels/val')

except OSError:
    print('Errore nella creazione della cartella labels/val')

try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/labels/train'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/labels/train')

except OSError:
    print('Errore nella creazione della cartella labels/train')


Attraverso degli array di appoggio dividiamo le labels dalle immagini, e settiamo le percentuali con cui fare lo split del dataset

In [23]:
#array che contiene i file
image = []
label = [] 

#path
all_data_path = '/content/yolov5/Olive_Detection_Tracking/dataset'
train_image_path = '/content/yolov5/Olive_Detection_Tracking/images/train'
val_image_path = '/content/yolov5/Olive_Detection_Tracking/images/val'
train_label_path ='/content/yolov5/Olive_Detection_Tracking/labels/train'
val_label_path = '/content/yolov5/Olive_Detection_Tracking/labels/val'


#percentuale per train, val
val_ratio = 0.2
train_ratio = 0.8

#inserisco file negli array corrispondenti
for (dirname, dirs, files) in os.walk(all_data_path):
    for filename in files:
        if filename.endswith('.txt'):
            label.append(filename)
        else:
            image.append(filename)

#numero di immagini per train e per val
num_train = int(len(image)*train_ratio)
num_val = int(len(image)*val_ratio)

print("Immagini per il training: ", num_train)
print("Immagini per validation: ", num_val)



Immagini per il training:  0
Immagini per validation:  0


Con la funzione random separiamo in modo casuale le immagini usate per il traing da quelle usate per la validation
Successivamente inseriamo le rispettive labels nella directory per il training oppure per la validation

In [24]:
#inserisco image e label in train
for i in range(num_train):
   file_image = random.choice(image)
   file_image_name = file_image.split('.')[0]
   file_label =  file_image_name+'.txt'

   #sposto i file nella cartella train
   shutil.copy(os.path.join(all_data_path, file_image), os.path.join(train_image_path, file_image))
   shutil.copy(os.path.join(all_data_path, file_label), os.path.join(train_label_path, file_label))

   #rimuovo i file da arrays
   image.remove(file_image)
   label.remove(file_label)
    

#inserisco image e label in val  
for i in range(num_val):
    file_image =random.choice(image) # get name of random image from origin dir
    file_image_name = file_image.split('.')[0]
    file_label = file_image_name+'.txt'

    #move both files into train dir
    shutil.copy(os.path.join(all_data_path, file_image), os.path.join(val_image_path, file_image))
    shutil.copy(os.path.join(all_data_path, file_label), os.path.join(val_label_path, file_label))
    
    #rimuovo i file da arrays
    image.remove(file_image)
    label.remove(file_label)


Inseriamo file yaml nella directory data

In [ ]:
source = 'Olive_Detection_Tracking/dataset.yaml'
destination = 'data/dataset.yaml'
shutil.copy(source,destination)

## **ADDESTRAMENTO SOURCE DETECTOR**

Usiamo Weights & Biases per mostrare i risultati

In [ ]:
#Weights & Biases
%pip install -q wandb
import wandb
wandb.login()

Avviamo l'addestramento 

In [ ]:
!python train.py --img 416 --batch 16 --epochs 30 --data dataset.yaml --weights yolov5s.pt --cache 

Controlliamo i risultati stampandoli a schermo

In [ ]:
Image.open('runs/train/exp/train_batch0.jpg')

In [ ]:
Image.open('runs/train/exp/val_batch0_labels.jpg')

In [ ]:
Image.open('runs/train/exp/val_batch0_pred.jpg')

In [ ]:

file_csv=pd.read_csv('runs/train/exp/results.csv')
file_csv.head(-1)


In [ ]:
Image.open('runs/train/exp/results.png')

## **INFERENZA**




Facciamo il download del weights migliore cosi da salvarlo in locale e poterlo riutilizzare per inferences future

In [ ]:
from google.colab import files
files.download('runs/train/exp/weights/best.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Facciamo l'inferenza, tramite detec.py, sui frame video cosi da ottenere delle labels che poi verrano usate per generare le pseudo labels. L' inferenza viene fatta con una threshold di 0.40.
I risultati sono salvati in runs/detect/exp

In [ ]:
!python detect.py --source Olive_Detection_Tracking/keyframe_video --data data/dataset.yaml --weights runs/train/exp/weights/best.pt --save-txt --conf-thres 0.40

Creiamo la zip della cartella contenente le labels cosi da poter fare il download e salvarle in locale.

In [ ]:
!zip -r labels.zip runs/detect/exp/labels

In [ ]:
from google.colab import files
files.download("labels.zip")


##**CREAZIONE DATASET TARGET DETECTOR**

Caricare la cartella contenente le pseudo labels("Results__PseudoLabels") generate tramite i software contenuti nella seguente repo https://github.com/Lio320/GraphTracking.git 

In [4]:
!git clone https://github.com/GretaDanesi18/Olive_Detection_Tracking.git

Cloning into 'Olive_Detection_Tracking'...
remote: Enumerating objects: 2912, done.
remote: Total 2912 (delta 0), reused 0 (delta 0), pack-reused 2912
Receiving objects: 100% (2912/2912), 1.60 GiB | 27.69 MiB/s, done.
Resolving deltas: 100% (38/38), done.
Checking out files: 100% (2809/2809), done.


In [5]:
try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/Psuedo_Labels/images/val'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/Psuedo_Labels/images/val')

except OSError:
    print('Errore nella creazione della cartella images/val')

try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/Psuedo_Labels/images/train'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/Psuedo_Labels/images/train')

except OSError:
    print('Errore nella creazione della cartella images/train')

try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/Psuedo_Labels/labels/val'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/Psuedo_Labels/labels/val')

except OSError:
    print('Errore nella creazione della cartella labels/val')

try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/Psuedo_Labels/labels/train'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/Psuedo_Labels/labels/train')

except OSError:
    print('Errore nella creazione della cartella labels/train')


Faccio lo split del target dataset cosi da ottenere il dataset per il training e per la validation

In [ ]:
#array che contiene i file
image = []
label = [] 

#path
all_data_path = '/content/yolov5/Olive_Detection_Tracking/psuedo_labels_dataset'
train_image_path = '/content/yolov5/Olive_Detection_Tracking/Psuedo_Labels/images/train'
val_image_path = '/content/yolov5/Olive_Detection_Tracking/Psuedo_Labels/images/val'
train_label_path ='/content/yolov5/Olive_Detection_Tracking/Psuedo_Labels/labels/train'
val_label_path = '/content/yolov5/Olive_Detection_Tracking/Psuedo_Labels/labels/val'


#percentuale per train, val
val_ratio = 0.2
train_ratio = 0.8

#inserisco file negli array corrispondenti
for (dirname, dirs, files) in os.walk(all_data_path):
    for filename in files:
        if filename.endswith('.txt'):
            label.append(filename)
        else:
            image.append(filename)

#numero di immagini per train e per val
num_train = int(len(image)*train_ratio)
num_val = int(len(image)*val_ratio)

print("Immagini per il training: ", num_train)
print("Immagini per validation: ", num_val)

In [7]:
#inserisco image e label in train
for i in range(num_train):
   file_image = random.choice(image)
   file_image_name = file_image.split('.')[0]
   file_label =  file_image_name+'.txt'

   #sposto i file nella cartella train
   shutil.copy(os.path.join(all_data_path, file_image), os.path.join(train_image_path, file_image))
   shutil.copy(os.path.join(all_data_path, file_label), os.path.join(train_label_path, file_label))

   #rimuovo i file da arrays
   image.remove(file_image)
   label.remove(file_label)
    

#inserisco image e label in val  
for i in range(num_val):
    file_image =random.choice(image) # get name of random image from origin dir
    file_image_name = file_image.split('.')[0]
    file_label = file_image_name+'.txt'

    #move both files into train dir
    shutil.copy(os.path.join(all_data_path, file_image), os.path.join(val_image_path, file_image))
    shutil.copy(os.path.join(all_data_path, file_label), os.path.join(val_label_path, file_label))
    
    #rimuovo i file da arrays
    image.remove(file_image)
    label.remove(file_label)

Cambiare i path all' interno del file target_dataset.yaml in accordo con il proprio dataset

In [ ]:
#inserisco target_dataset.yal nella directory 'data'
source = 'Olive_Detection_Tracking/target_dataset.yaml'
destination = 'data/target_dataset.yaml'
shutil.copy(source,destination)

In [ ]:
os.getcwd()

## **ADDESTRAMENTO TARGET DETECTOR**

In [ ]:
#Weights & Biases
%pip install -q wandb
import wandb
wandb.login()

Avviamo l'addestramento

In [ ]:
!python train.py --img 416 --batch 16 --epochs 30 --data target_dataset.yaml --weights yolov5s.pt --cache 